# Creating A simple Agent

# 1. Define LLM

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from dotenv import load_dotenv 
import os 

# configures
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=os.getenv("GOOGLE_API_KEY"))

# 2. Define Custom Tools


In [4]:
def get_weather(city: str) -> str:
    response = f"The weather in {city} is sunny, 25°C."
    return response

# 3. Create tool list

In [5]:
tools = [
    Tool(
        name="Weather",
        func=get_weather,
        description="Use this tool to get the weather of a given city"
    )
]

# 4. Initialize agent

In [6]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # lets LLM decide when to call tools
    verbose=True
)

C:\Users\Noor Saeed\AppData\Local\Temp\ipykernel_19116\2403052513.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


#  5. Run query


In [7]:
response = agent.run("What's the weather in New York?")
print(response)

C:\Users\Noor Saeed\AppData\Local\Temp\ipykernel_19116\2716904819.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("What's the weather in New York?")




> Entering new AgentExecutor chain...
Action: Weather
Action Input: New York
Observation: The weather in New York is sunny, 25°C.
Thought:Final Answer: The weather in New York is sunny, 25°C.

> Finished chain.
The weather in New York is sunny, 25°C.


In [ ]:
response = agent.run("What's the weather in New York?")
print(response)

# Working With Multiple Tools

In [10]:
# Tool 1: Weather
def get_weather(city: str) -> str:
    # Dummy weather details
    condition = "Sunny"
    temperature = "25°C"
    humidity = "60%"
    wind = "12 km/h"
    air_quality = "Good (AQI 45)"
    uv_index = "Moderate (5)"
    
    return (
        f"Weather Report for {city}:\n"
        f"- Condition: {condition}\n"
        f"- Temperature: {temperature}\n"
        f"- Humidity: {humidity}\n"
        f"- Wind: {wind}\n"
        f"- Air Quality: {air_quality}\n"
        f"- UV Index: {uv_index}"
    )


weather_tool = Tool(
    name="Weather",
    func=get_weather,
    description="Get the current weather of a city."
)

In [12]:
# Tool 2: Database query (example)
def query_database(query: str) -> str:
    # Dummy metadata
    execution_time = "0.023s"
    rows_returned = 3
    
    # Dummy table rows
    results = [
        {"id": 1, "name": "Alice", "age": 29, "city": "Paris"},
        {"id": 2, "name": "Bob", "age": 34, "city": "London"},
        {"id": 3, "name": "Charlie", "age": 41, "city": "New York"},
    ]
    
    # Format results
    rows_text = "\n".join(
        [f"- ID: {r['id']}, Name: {r['name']}, Age: {r['age']}, City: {r['city']}" for r in results]
    )
    
    return (
        f"Query executed: {query}\n"
        f"Execution time: {execution_time}\n"
        f"Rows returned: {rows_returned}\n"
        f"Results:\n{rows_text}"
    )


database_tool = Tool(
    name="Database",
    func=query_database,
    description="Use this tool to query the internal database."
)

In [ ]:
# Tool 3: Web Search Tool
def web_search(query: str) -> str:
    # Dummy data dictionary
    dummy_results = {
        "Python tutorials": [
            "Learn Python: Official Python Tutorial",
            "Python for Beginners - W3Schools",
            "Python Crash Course - freeCodeCamp"
        ],
        "AI news": [
            "Latest AI breakthroughs - TechCrunch",
            "AI in 2025: What's coming next? - Wired",
            "OpenAI releases GPT-5 updates - The Verge"
        ],
        "New York weather": [
            "Weather in New York: Sunny, 25°C",
            "New York forecast: High 27°C, Low 19°C",
            "NYC weather updates today - Weather.com"
        ]
    }
    
    # Return dummy results if available, else a default message
    results = dummy_results.get(query, [f"No dummy results found for '{query}'"])
    
    # Format the results nicely
    formatted_results = "\n".join([f"{i+1}. {r}" for i, r in enumerate(results)])
    return f"Top search results for '{query}':\n{formatted_results}"

# Define the tool
web_tool = Tool(
    name="WebSearch",
    func=web_search,
    description="Use this tool to search the web for information (dummy data)."
)


In [13]:
# List of tools
tools = [weather_tool, database_tool, web_tool]

In [14]:
# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,  
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [15]:
# Run the agent
response = agent.run("What's the weather in New York and also search for Python tutorials?")
print(response)



> Entering new AgentExecutor chain...
Action: Weather
Action Input: New York
Observation: Weather Report for New York:
- Condition: Sunny
- Temperature: 25°C
- Humidity: 60%
- Wind: 12 km/h
- Air Quality: Good (AQI 45)
- UV Index: Moderate (5)
Thought:Action: WebSearch
Action Input: Python tutorials
Observation: Top search results for 'Python tutorials':
1. Learn Python: Official Python Tutorial
2. Python for Beginners - W3Schools
3. Python Crash Course - freeCodeCamp
Thought:I now know the final answer
Final Answer: The weather in New York is sunny with a temperature of 25°C, 60% humidity, and 12 km/h wind. The air quality is good (AQI 45) and the UV index is moderate (5). For Python tutorials, you can check out the Official Python Tutorial, W3Schools' Python for Beginners, or freeCodeCamp's Python Crash Course.

> Finished chain.
The weather in New York is sunny with a temperature of 25°C, 60% humidity, and 12 km/h wind. The air quality is good (AQI 45) and the UV index is moderate

In [16]:
response = agent.run("Query the employees table")
print(response)



> Entering new AgentExecutor chain...
Action: Database
Action Input: SELECT * FROM employees
Observation: Query executed: SELECT * FROM employees
Execution time: 0.023s
Rows returned: 3
Results:
- ID: 1, Name: Alice, Age: 29, City: Paris
- ID: 2, Name: Bob, Age: 34, City: London
- ID: 3, Name: Charlie, Age: 41, City: New York
Thought:I now know the final answer
Final Answer: I have queried the employees table. The results are:
- ID: 1, Name: Alice, Age: 29, City: Paris
- ID: 2, Name: Bob, Age: 34, City: London
- ID: 3, Name: Charlie, Age: 41, City: New York

> Finished chain.
I have queried the employees table. The results are:
- ID: 1, Name: Alice, Age: 29, City: Paris
- ID: 2, Name: Bob, Age: 34, City: London
- ID: 3, Name: Charlie, Age: 41, City: New York
